In [1]:
import pandas as pd
from sample_weight_audit import paired_test
import threadpoolctl

threadpoolctl.threadpool_limits(limits=1)
import warnings
warnings.filterwarnings("ignore")

from sklearn import utils
from inspect import signature

ests = utils.all_estimators(type_filter='classifier')

df = []

for est in ests[1:]:
    est = est[1]
    if 'random_state' in signature(est).parameters and 'sample_weight' in signature(est.fit).parameters:
        if est.__name__ == "sklearn.linear_model._logistic.LogisticRegressionCV":
            continue
        ## record in dataframe name of estimator, and all pvalues and minimum pvalue and average pvalue
        #try:
        df.append(paired_test(est, n_features = 10, test='welch', max_seed=20,
                train_size = 300, n_samples_per_cv_group=500,
                rep_test_size = 20, max_repeats=5, n_classes=8,
                correct_threshold=True, equal_var=False,issparse=False))
        #except ValueError:
        #    print(est, "with different random state led to the same predictions")
        
df = pd.DataFrame(df)
df.sort_values(by=['min_p_value'])



100%|██████████| 20/20 [00:01<00:00, 11.05it/s]


Finished looping till the maximum random state, 20 for estimator <class 'sklearn.ensemble._bagging.BaggingClassifier'> in ---- 1.8797709941864014 s---
Minimum p-values:  5.569062975843453e-05


100%|██████████| 20/20 [00:00<00:00, 157.13it/s]


Finished looping till the maximum random state, 20 for estimator <class 'sklearn.tree._classes.DecisionTreeClassifier'> in ---- 0.15209317207336426 s---
Minimum p-values:  1.0


100%|██████████| 20/20 [00:00<00:00, 2354.37it/s]


...

Finished looping till the maximum random state, 20 for estimator <class 'sklearn.tree._classes.ExtraTreeClassifier'> in ---- 0.06030106544494629 s---
Minimum p-values:  1.0


100%|██████████| 20/20 [00:03<00:00,  6.14it/s]


Finished looping till the maximum random state, 20 for estimator <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> in ---- 3.355586051940918 s---
Minimum p-values:  1.0


 25%|██▌       | 5/20 [00:10<00:30,  2.06s/it]


KeyboardInterrupt: 